**PubMed Searcher Demo**

This notebook is intended to demonstrate how to use the `PubMedSearcher` class to search, download, and explore references from PubMed queries.

To create instance of the PubMedSearcher class, just provide a search string, an email, and an optional dataframe that will be used to store the results.

The important methods of the class are:
- `search`: to search for references in PubMed. This will populate the df attribute with results, including the title, abstract, and other metadata.
- `download_articles`: to download PDFs for the articles found in the search. Uses open access links when available, but can fallback to Sci-Hub.
- `fetch_references`: Fetches references from each article in the search results using APIs. Returns a JSON object with the references.
- `fetch_cited_by`: Fetches articles that cite each article in the search results using APIs. Returns a JSON object with the references.
- `download_xml_fulltext`: Downloads the full text of the articles in XML format. Not commonly available, but works for articles in the PubMed open-access subset.

In [1]:
from calvin_utils.gpt_sys_review.pubmed_utils import PubMedSearcher

# Example usage
search_string = "brain lesions"
email = "lesion_bank@gmail.com"
searcher = PubMedSearcher(search_string=search_string)
searcher.search(count=5, min_date=2018, order_by='relevance')

In [2]:
searcher.fetch_references()
searcher.fetch_cited_by()
searcher.download_articles()
searcher.df

Fetching Cited By: 100%|██████████| 5/5 [00:02<00:00,  1.84it/s]


,title,authors,first_author,abstract,publication_date,publication_year,journal_info,doi,pmcid,pmid,...,pdf_url_4,europe_pmc_url,keywords,article_type,country,language,references,cited_by,download_complete,pdf_filepath
0,Transcranial Doppler ultrasound in neurocritic...,"Blanco, Pablo; Abdo-Cuza, Anselmo",Blanco,Multimodality monitoring is a common practice ...,"{'Year': '2018', 'Month': '02', 'Day': '10'}",2018,Journal of ultrasound,10.1007/s40477-018-0282-9,PMC5845939,29429015,...,None,https://europepmc.org/articles/pmc5845939,Brain injury; Critical care; Point-of-care-ult...,Journal Article; Review,Italy,eng,"{'version': '6.9', 'hitCount': 32, 'request': ...","{'version': '6.9', 'hitCount': 21, 'request': ...",Complete,downloads/0_Blanco_2018/Blanco_2018.pdf
1,Magnetic Resonance Imaging in Multiple Sclerosis.,"Hemond, Christopher C; Bakshi, Rohit",Hemond,Since its technical development in the early 1...,"{'Year': '2018', 'Month': '05', 'Day': '01'}",2018,Cold Spring Harbor perspectives in medicine,10.1101/cshperspect.a028969,PMC5932576,29358319,...,None,https://europepmc.org/articles/pmc5932576,,Journal Article; Review,United States,eng,"{'version': '6.9', 'hitCount': 195, 'request':...","{'version': '6.9', 'hitCount': 33, 'request': ...",Complete,downloads/1_Hemond_2018/Hemond_2018.pdf
2,Genome-wide association studies of brain imagi...,"Elliott, Lloyd T; Sharp, Kevin; Alfaro-Almagro...",Elliott,The genetic architecture of brain structure an...,"{'Year': '2018', 'Month': '10', 'Day': '10'}",2018,Nature,10.1038/s41586-018-0571-7,PMC6786974,30305740,...,None,https://europepmc.org/articles/pmc6786974,,"Journal Article; Research Support, Non-U.S. Gov't",England,eng,"{'version': '6.9', 'hitCount': 51, 'request': ...","{'version': '6.9', 'hitCount': 272, 'request':...",Complete,downloads/2_Elliott_2018/Elliott_2018.pdf
3,Local sleep.,"Krueger, James M; Nguyen, Joseph T; Dykstra-Ai...",Krueger,The historic sleep regulatory paradigm invokes...,"{'Year': '2018', 'Month': '11', 'Day': '12'}",2018,Sleep medicine reviews,10.1016/j.smrv.2018.10.001,PMC6351167,30502497,...,None,https://europepmc.org/articles/pmc6351167,Bottom-up sleep; Cortical column sleep; In vit...,"Journal Article; Research Support, N.I.H., Ext...",England,eng,"{'version': '6.9', 'hitCount': 110, 'request':...","{'version': '6.9', 'hitCount': 41, 'request': ...",Complete,downloads/3_Krueger_2018/Krueger_2018.pdf
4,Somatosensory deficits.,"Klingner, Carsten M; Witte, Otto W",Klingner,The analysis and interpretation of somatosenso...,{},None,Handbook of clinical neurology,10.1016/B978-0-444-63622-5.00009-7,,29519458,...,None,None,deficits; parietal lobe; pathways; rehabilitat...,Journal Article; Review,Netherlands,eng,[{'key': '10.1016/B978-0-444-63622-5.00009-7_b...,None,Unavailable,None
